In [216]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import scipy.stats as st
from scipy.stats import linregress
import datetime
from datetime import datetime, timedelta



In [217]:
# Import the Data CSV Files and clean the data


SP500 = pd.read_csv ("S&P 500 from 1989 - Now Yahoo FInance.csv")
ten_yr_bond= pd.read_csv("^TNX 10 year yield from 1989.csv")
gold=pd.read_csv("GOLDFILE FOR PROJECT.csv")
vix=pd.read_csv("VIX Data from 1990.csv")

# S&P sector data w/o XLC & XLRE sectors
materials_sp500=pd.read_csv("11 S&P Funds/XLB 1998.csv")
energy_sp500=pd.read_csv("11 S&P Funds/XLE 1998-Now.csv")
financials_sp500= pd.read_csv("11 S&P Funds/XLF 1998.csv")
industrial_sp500=pd.read_csv("11 S&P Funds/XLI 1998.csv")
tech_sp500=pd.read_csv("11 S&P Funds/XLK From 1998.csv")
consumerstaple_sp500=pd.read_csv("11 S&P Funds/XLP from 1998.csv")
utilities_sp500=pd.read_csv("11 S&P Funds/XLU From 1998.csv")
health_sp500=pd.read_csv("11 S&P Funds/XLV From 1998- Now.csv")
consumerdiscretionary_sp500=pd.read_csv("11 S&P Funds/XLY from 1998.csv")

vix

,Date,Open,Close
0,1/1/1990,18.190000,18.190000
1,2/1/1990,24.870001,21.990000
2,3/1/1990,21.900000,19.730000
3,4/1/1990,22.840000,19.520000
4,5/1/1990,18.430000,17.370001
...,...,...,...
377,6/1/2021,17.240000,15.830000
378,7/1/2021,15.620000,18.240000
379,8/1/2021,18.160000,16.480000
380,9/1/2021,16.059999,23.139999


In [218]:
# Make Comparitive Line Chart for comparing the following: 
# Year over year % change in price of gold vs S&P 500 from 1990-2020
# Year over Year Change in VIX vs S&P 500 from 1990-2020
# Year over Year % change in S&P 500 and 10 Yr Treasure Rate
SP500

,Date,Open,Close*
0,11/1/1989,340.36,345.99
1,12/1/1989,346.01,353.4
2,1/1/1990,353.4,329.08
3,2/1/1990,329.08,331.89
4,3/1/1990,331.89,339.94
...,...,...,...
379,6/1/2021,"4,216.52","4,297.50"
380,7/1/2021,"4,300.73","4,395.26"
381,8/1/2021,"4,406.86","4,522.68"
382,9/1/2021,"4,528.80","4,307.54"


In [229]:
#Clean the following DFs: SP500, VIX, Ten Year Treasury, Gold.  Merge all 4 of these DFs and then pull data from 

#Prior to Merging Data change column headers
vix=vix.rename(columns={"Open":"VIX Open", "Close": "VIX Close"})
ten_yr_bond=ten_yr_bond.rename(columns={"Open":"10 Yr Bond Open", "Close": "10 Yr Bond Close"})
gold=gold.rename(columns={"date":"Date","price":"Gold Close Price"})
SP500=SP500.rename(columns={"Open":"S&P 500 Open", "Close*": "S&P 500 Close"})

# Merge the Data for column header

Merged_df =SP500.merge(vix, how="inner", on="Date")
Merged_df=Merged_df.merge(gold, how="inner", on="Date")
Merged_df=Merged_df.merge(ten_yr_bond, how="inner", on="Date")

Merged_df['Date']=pd.to_datetime(Merged_df['Date'], format='%m/%d/%Y')
Merged_df['Year'] = Merged_df['Date'].dt.year
Merged_df['Month'] = Merged_df['Date'].dt.month
Merged_df['Day'] = Merged_df['Date'].dt.day



Merged_df

Merged_df.loc[(Merged_df['Month'] == 1)]

,Date,S&P 500 Open,S&P 500 Close,VIX Open,VIX Close,Gold Close Price,10 Yr Bond Open,10 Yr Bond Close,Year,Month,Day
0,1990-01-01,353.4,329.08,18.190000,18.190000,410.118,7.940,8.430,1990,1,1
12,1991-01-01,330.2,343.93,26.620001,20.910000,384.591,7.970,8.030,1991,1,1
24,1992-01-01,417.03,408.78,19.690001,17.400000,354.436,6.780,7.310,1992,1,1
36,1993-01-01,435.7,438.78,12.960000,12.420000,328.993,6.600,6.390,1993,1,1
48,1994-01-01,466.51,481.61,11.280000,10.630000,387.11,5.796,5.638,1994,1,1
60,1995-01-01,459.21,470.42,14.090000,11.960000,378.738,7.822,7.593,1995,1,1
72,1996-01-01,615.93,636.02,13.390000,12.530000,398.695,5.583,5.581,1996,1,1
84,1997-01-01,740.74,786.16,21.540001,19.469999,355.025,6.407,6.503,1997,1,1
96,1998-01-01,970.43,980.28,24.340000,21.469999,289.264,5.731,5.512,1998,1,1
108,1999-01-01,"1,229.23","1,279.64",25.379999,26.250000,287.333,4.638,4.653,1999,1,1


In [204]:
# Line Graph 1: Create a Line Graph of the year over year performance for S&P vs Gold from 1990-2020

# Get yearly S&P Data

S=Merged_df.iloc[0,2]

'329.08'

AttributeError: 'str' object has no attribute 'day_name'